# Homework 5 - Deployment
The goal of this homework is to host the prediction service in a docker container.

## Question 1:
What version of `pipenv` is installed?

In [1]:
# I've installed uv using the requirements.txt file
# in the docker container hosting my jupyter notebook
!uv --version

uv 0.9.5


## Question 2:
What's the first hash for scikit-learn in `Pipfile.lock`?

In [2]:
# I will only run this once, since the lockfile will be committed
#!uv init
#!uv add scikit-learn==1.6.1

Initialized project `05`
Using CPython 3.13.7 interpreter at: /opt/conda/bin/python3
Creating virtual environment at: .venv
Resolved 6 packages in 790ms                                         
Prepared 5 packages in 2.86s                                             
░░░░░░░░░░░░░░░░░░░░ [0/5] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 5 packages in 4.30s                               
 + joblib==1.5.2
 + numpy==2.3.4
 + scikit-learn==1.6.1
 + scipy==1.16.3
 + threadpoolctl==3.6.0


In [9]:
import tomllib
with open('uv.lock', 'rb') as f:
    data = tomllib.load(f)
    scikit_learn_pkg = [pkg for pkg in data['package'] if pkg['name'] == 'scikit-learn']
    print(len(scikit_learn_pkg))
    print(scikit_learn_pkg[0]['sdist']['hash'])

1
sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e


## Models
Download the models that were prepared for this homework.

In [10]:
#!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

--2025-10-29 10:28:55--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-29 10:28:56--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin’

pipeline_v1.bin

In [12]:
!md5sum pipeline_v1.bin

7d17d2e4dfbaf1e408e1a62e6e880d49  pipeline_v1.bin


## Question 3
Using pickle to unpack the provided model, what's the probability that this lead will convert?
```json
{
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}
```

In [14]:
import pickle

with open('pipeline_v1.bin', 'rb') as f_in:
    pipeline = pickle.load(f_in)

    customer = {
        "lead_source": "paid_ads",
        "number_of_courses_viewed": 2,
        "annual_income": 79276.0
    }

    prob = pipeline.predict_proba(customer)[0, 1]

    print(prob)

0.5336072702798061


## Question 4
Hosting the model as web service, what's the probability that this client will get a subscription?
```json
{
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
```